In [11]:
#!wget https://raw.githubusercontent.com/circulosmeos/gdown.pl/master/gdown.pl

In [12]:
#!./gdown.pl https://drive.google.com/file/d/1lhwcRhxzJ6BC5SUSRALRdCNKo4B4rwa1 Dataset.zip

In [114]:
#!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/dataset1/
   creating: Dataset/dataset1/forge/
  inflating: Dataset/dataset1/forge/02100001.png  
  inflating: Dataset/dataset1/forge/02100002.png  
  inflating: Dataset/dataset1/forge/02100003.png  
  inflating: Dataset/dataset1/forge/02100004.png  
  inflating: Dataset/dataset1/forge/02100005.png  
  inflating: Dataset/dataset1/forge/02100006.png  
  inflating: Dataset/dataset1/forge/02100007.png  
  inflating: Dataset/dataset1/forge/02100008.png  
  inflating: Dataset/dataset1/forge/02100009.png  
  inflating: Dataset/dataset1/forge/02100010.png  
  inflating: Dataset/dataset1/forge/02100011.png  
  inflating: Dataset/dataset1/forge/02100012.png  
  inflating: Dataset/dataset1/forge/02101001.png  
  inflating: Dataset/dataset1/forge/02101002.png  
  inflating: Dataset/dataset1/forge/02101003.png  
  inflating: Dataset/dataset1/forge/02101004.png  
  inflating: Dataset/dataset1/forge/02101005.png  
  inflating: Dataset/dataset1/forge/0210100

  inflating: Dataset/dataset2/real/00305003.png  
  inflating: Dataset/dataset2/real/00401004.png  
  inflating: Dataset/dataset2/real/00402004.png  
  inflating: Dataset/dataset2/real/00403004.png  
  inflating: Dataset/dataset2/real/00404004.png  
  inflating: Dataset/dataset2/real/00405004.png  
  inflating: Dataset/dataset2/real/00501005.png  
  inflating: Dataset/dataset2/real/00502005.png  
  inflating: Dataset/dataset2/real/00503005.png  
  inflating: Dataset/dataset2/real/00504005.png  
  inflating: Dataset/dataset2/real/00505005.png  
  inflating: Dataset/dataset2/real/00601006.png  
  inflating: Dataset/dataset2/real/00602006.png  
  inflating: Dataset/dataset2/real/00603006.png  
  inflating: Dataset/dataset2/real/00604006.png  
  inflating: Dataset/dataset2/real/00605006.png  
  inflating: Dataset/dataset2/real/00701007.png  
  inflating: Dataset/dataset2/real/00702007.png  
  inflating: Dataset/dataset2/real/00703007.png  
  inflating: Dataset/dataset2/real/00704007.png  


  inflating: Dataset/dataset3/real/02201022.png  
  inflating: Dataset/dataset3/real/02202022.png  
  inflating: Dataset/dataset3/real/02203022.png  
  inflating: Dataset/dataset3/real/02204022.png  
  inflating: Dataset/dataset3/real/02205022.png  
  inflating: Dataset/dataset3/real/02301023.png  
  inflating: Dataset/dataset3/real/02302023.png  
  inflating: Dataset/dataset3/real/02303023.png  
  inflating: Dataset/dataset3/real/02304023.png  
  inflating: Dataset/dataset3/real/02305023.png  
  inflating: Dataset/dataset3/real/02401024.png  
  inflating: Dataset/dataset3/real/02402024.png  
  inflating: Dataset/dataset3/real/02403024.png  
  inflating: Dataset/dataset3/real/02404024.png  
  inflating: Dataset/dataset3/real/02405024.png  
  inflating: Dataset/dataset3/real/02501025.png  
  inflating: Dataset/dataset3/real/02502025.png  
  inflating: Dataset/dataset3/real/02503025.png  
  inflating: Dataset/dataset3/real/02504025.png  
  inflating: Dataset/dataset3/real/02505025.png  


  inflating: Dataset/dataset4/real1/09301093.png  
  inflating: Dataset/dataset4/real1/09302093.png  
  inflating: Dataset/dataset4/real1/09303093.png  
  inflating: Dataset/dataset4/real1/09304093.png  
  inflating: Dataset/dataset4/real1/09305093.png  


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import imread,io

In [4]:
import pickle

In [5]:
from scipy.misc import imread,imsave
from preprocess.normalize import preprocess_signature
import architecture
from cnn_model import CNNModel



WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
/tmp/try_flags_g3b78W.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:08:00.0)


In [7]:
model_weight_path = 'models/weights.pkl'
model = CNNModel(architecture, model_weight_path)

## Dataset Loading

In [8]:
database=[]
test=[]
database=(os.listdir('DataBase/'))

test=(os.listdir('Test/'))


## Preprocessing

In [9]:
import cv2
import numpy as np
from scipy import ndimage
from scipy.misc import imresize

def normalize_image(img, size=(840, 1360)):

    max_r, max_c = size

    blur_radius = 2
    blurred_image = ndimage.gaussian_filter(img, blur_radius)

    threshold, binarized_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    r, c = np.where(binarized_image == 0)
    r_center = int(r.mean() - r.min())
    c_center = int(c.mean() - c.min())

    cropped = img[r.min(): r.max(), c.min(): c.max()]

    img_r, img_c = cropped.shape

    r_start = max_r // 2 - r_center
    c_start = max_c // 2 - c_center


    if img_r > max_r:
        r_start = 0
        difference = img_r - max_r
        crop_start = difference // 2
        cropped = cropped[crop_start:crop_start + max_r, :]
        img_r = max_r
    else:
        extra_r = (r_start + img_r) - max_r
        if extra_r > 0:
            r_start -= extra_r
        if r_start < 0:
            r_start = 0

    if img_c > max_c:
        c_start = 0
        difference = img_c - max_c
        crop_start = difference // 2
        cropped = cropped[:, crop_start:crop_start + max_c]
        img_c = max_c
    else:
        extra_c = (c_start + img_c) - max_c
        if extra_c > 0:
            c_start -= extra_c
        if c_start < 0:
            c_start = 0

    normalized_image = np.ones((max_r, max_c), dtype=np.uint8) * 255
    normalized_image[r_start:r_start + img_r, c_start:c_start + img_c] = cropped

    normalized_image[normalized_image > threshold] = 255

    return normalized_image



In [10]:
import PIL
from PIL import Image

def res(path,nm):
    mywidth = 500
    img = Image.open(path).convert('L')
    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img.save(path)

In [11]:
emd_database=defaultdict(list)
emd_test=defaultdict(list)
path_db="DataBase/"
path_test="Test/"
for i in database:
    path=path_db+i
    temp = cv2.imread(path,0)
    temp=preprocess_signature(temp,(876,1240))
    emd=model.get_feature_vector(temp)
    emd_database[i[0:6]].append(emd)
for i in test:
    path=path_test+i
    temp = cv2.imread(path,0)
    temp=preprocess_signature(temp,(876,1240))
    emd=model.get_feature_vector(temp)
    emd_test[i[0:6]].append(emd)



In [14]:
fle_name=defaultdict(list)

In [15]:
for i in test:
    fle_name[i[0:6]].append(i)

In [16]:
lst2=emd_database.keys()

In [17]:

from sklearn.metrics import f1_score, accuracy_score

distances = defaultdict(list) 

print("Gen Started")
for i in lst2:
    for k in range(len(emd_test[i])):
            for l in range(len(emd_database[i])):
                dist=100-np.linalg.norm(emd_database[i][l]-emd_test[i][k])
                distances[fle_name[i][k]].append(dist)

        







Gen Started


In [21]:
pwd

u'/home1/tirthankar/SUMMER_INTERN/NOVELTY/Abhishek/SIGV/Untitled Folder/sigver_wiwd'

In [18]:
for i in distances.keys():
    for j in distances[i]:
        print(i,j)

('U11010_264.jpg', 88.8787899017334)
('U11010_264.jpg', 89.2483606338501)
('U11010_264.jpg', 90.61300373077393)
('U11006_12.jpg', 75.83515739440918)
('U11006_12.jpg', 80.07926368713379)
('U11006_12.jpg', 73.45742416381836)
('U11014_6.jpg', 88.48555278778076)
('U11014_6.jpg', 75.94696235656738)
('U11014_6.jpg', 76.44581031799316)
('U11027_205.jpg', 76.3985481262207)
('U11027_205.jpg', 74.38416481018066)
('U11027_205.jpg', 81.93831634521484)
('U11010_363.jpg', 89.45132064819336)
('U11010_363.jpg', 88.34843921661377)
('U11010_363.jpg', 93.76168632507324)
('U11003_333.jpg', 78.78490447998047)
('U11003_333.jpg', 80.84343147277832)
('U11003_333.jpg', 73.49508476257324)
('U11011_22.jpg', 83.25394821166992)
('U11011_22.jpg', 85.70793151855469)
('U11011_22.jpg', 88.19262599945068)
('U11015_94.jpg', 85.88261222839355)
('U11015_94.jpg', 88.44489002227783)
('U11015_94.jpg', 84.75915813446045)
('U11025_217.jpg', 86.80396747589111)
('U11025_217.jpg', 85.7341480255127)
('U11025_217.jpg', 88.791853904

In [19]:
fd=open("White_Hats.csv","w+")
fd.write("File Name,Forgery\n")
for i in distances.keys():
    av=0.0
    for j in distances[i]:
        av+=j
    av=av/3
    if(av>=85):
        fd.write(str(i)+","+"NO")
    else:
        fd.write(str(i)+","+"YES")
    fd.write("\n")
fd.close()

In [20]:
def viz():
    thresholds = np.arange(0, 1, .0001)

    f1_scores = [f1_score(identical, distances < t) for t in thresholds]
    acc_scores = [accuracy_score(identical, distances < t) for t in thresholds]

    opt_idx = np.argmax(f1_scores)
    opt_tau = thresholds[opt_idx]
    opt_acc = accuracy_score(identical, distances < opt_tau)
    plt.plot(thresholds, f1_scores, label='F1 score');
    plt.plot(thresholds, acc_scores, label='Accuracy');
    plt.axvline(x=opt_tau, linestyle='--', lw=1, c='lightgrey', label='Threshold')
    plt.xlabel('Distance threshold')
    plt.legend();
    print(opt_tau,opt_acc)